# Drum Transcriber (Omnizart Edition) Colab

Run this notebook to launch the **SOTA Omnizart-powered** Drum Transcriber Web UI.

**Note**: This notebook installs Omnizart from source to fix dependency issues on modern Colab runtimes.

In [ ]:
# @title 1. Setup Environment
# @markdown This cell installs necessary libraries and patches Omnizart dependencies.

import os

# 1. Clamp basic dependencies & Fix Build Environment
# Force PyYAML >= 6.0 because 5.4.1 fails to build on Python 3.10+
!pip install "PyYAML>=6.0" Cython
!pip install pretty_midi gradio yt-dlp librosa pandas numpy plotly scikit-learn soundfile madmom

# 2. Clone and Patch Omnizart (The PyPI version is too old for Colab)
if not os.path.exists("omnizart"):
    !git clone https://github.com/Music-and-Culture-Technology-Lab/omnizart.git

%cd omnizart

# Patch setup.py to relax strict versions
# 1. Allow newer PyYAML
!sed -i 's/PyYAML<6.0.0,>=5.3.1/PyYAML>=5.3.1/' setup.py
# Fallback: replace any PyYAML constraint if the above regex misses
!sed -i 's/PyYAML<6.0.0/PyYAML>=5.3.1/' setup.py

# 2. Relax Numba and Librosa
!sed -i 's/numba==0.48/numba>=0.48/' setup.py
!sed -i 's/librosa==0.8.0/librosa>=0.8.0/' setup.py
!sed -i 's/jsonschema<4.0.0,>=3.2.0/jsonschema>=3.2.0/' setup.py

# 3. Patch requirements.txt if it exists (it might dictate pip's behavior)
if os.path.exists("requirements.txt"):
    !sed -i 's/PyYAML<6.0.0/PyYAML>=5.3.1/' requirements.txt
    !sed -i 's/numba==0.48/numba>=0.48/' requirements.txt
    !sed -i 's/librosa==0.8.0/librosa>=0.8.0/' requirements.txt

# Install patched omnizart
!pip install .

%cd ..

# 3. Clone DrumTranscriber Repo
repo_url = "https://github.com/AgentHitmanFaris/DrumTranscriber.git"
repo_name = "DrumTranscriber"

if os.path.exists(repo_name):
    %cd {repo_name}
    !git pull
else:
    !git clone {repo_url}
    %cd {repo_name}

# Download Omnizart checkpoints via Python (avoid CLI path issues)
try:
    from omnizart.remote import download_checkpoints
    print("Downloading Omnizart Checkpoints...")
    download_checkpoints()
except ImportError:
    print("Omnizart import failed. Please check installation.")
except Exception as e:
    print(f"Error downloading checkpoints: {e}")


In [ ]:
# @title 2. Run App (Omnizart Mode)
# @markdown This will launch the Gradio interface using the Omnizart model.

launcher_code = """
import os
import gradio_app
from omnizart_wrapper import OmnizartWrapper

# Patch the load_model function to return OmnizartWrapper
def load_omnizart():
    print('Loading Omnizart Wrapper...')
    return OmnizartWrapper()

gradio_app.load_model = load_omnizart

# Run the demo
gradio_app.demo.launch(share=True)
"""

with open('run_omnizart.py', 'w') as f:
    f.write(launcher_code)

!python run_omnizart.py
